# architecture-12 ( Basic Regression Model Copy From 05 )

What's new:

1- Add keras tuner

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks

In [ ]:
df = pd.read_csv('datasets-13/XAGUSD-H1-rates.csv', sep='\t').dropna()

In [ ]:
# === Scaling Features ===
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[['<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>']])

close_scaler = MinMaxScaler()
scaled_close = close_scaler.fit_transform(df[['<CLOSE>']])

In [ ]:
# === Prepare sequences ===
def create_sequences(features, target, window, horizon):
    X, y = [], []
    for i in range(window, len(features) - horizon):
        X.append(features[i - window:i])
        y.append(target[i:i + horizon])
    return np.array(X), np.array(y)


WINDOW_SIZE = 60
FORECAST_HORIZON = 10

X, y = create_sequences(scaled, scaled_close, WINDOW_SIZE, FORECAST_HORIZON)

# === Train/test split (chronological) ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



In [ ]:
# === Manual time-based validation split ===
val_split_idx = int(len(X_train) * 0.8)
X_tune_train, X_tune_val = X_train[:val_split_idx], X_train[val_split_idx:]
y_tune_train, y_tune_val = y_train[:val_split_idx], y_train[val_split_idx:]

# === Define HyperModel ===
class LSTMHyperModel(HyperModel):
    def __init__(self, input_shape, output_dim):
        self.input_shape = input_shape
        self.output_dim = output_dim

    def build(self, hp):
        model = Sequential()

        # Tune LSTM units
        units = hp.Int('lstm_units', min_value=16, max_value=128, step=16)
        dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
        model.add(LSTM(units, input_shape=self.input_shape, dropout=dropout))

        # Optional extra dense layer
        if hp.Boolean('use_dense'):
            dense_units = hp.Int('dense_units', 16, 128, step=16)
            model.add(Dense(dense_units, activation='relu'))

        # Output layer
        model.add(Dense(self.output_dim))

        # Tune optimizer learning rate
        lr = hp.Float('lr', min_value=1e-4, max_value=1e-2, sampling='log')
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        return model


In [ ]:
# === Instantiate the Tuner ===
from keras_tuner.tuners import RandomSearch

tuner = RandomSearch(
    LSTMHyperModel(input_shape=(X_train.shape[1], X_train.shape[2]), output_dim=FORECAST_HORIZON),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='forex_lstm_tuning'
)


In [ ]:
# === Search the Best Hyperparameters ===
es_tuner = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
mc_tuner = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',  # for val_loss -- max for val_accuracy auto for...
    verbose=1
)


# === Run hyperparameter search ===
tuner.search(
    X_tune_train, y_tune_train,
    validation_data=(X_tune_val, y_tune_val),
    epochs=20,
    callbacks=[es_tuner, mc_tuner],
    verbose=1
)

In [ ]:
# === Retrieve and Train the Best Model ===
best_model = tuner.get_best_models(num_models=1)[0]

es_final = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
mc_final = callbacks.ModelCheckpoint(filepath='best_final_model.keras', monitor='val_loss', save_best_only=True)



# Optionally fine-tune more
history = best_model.fit(
    X_train, y_train,
    epochs=50,
    validation_split=0.2,  # Final fine-tuning can still use val_split for convenience
    callbacks=[es_final, mc_final],  # or define separately callbacks
    verbose=1
)

In [ ]:
# === Load the last 60 rows from a separate CSV file for prediction ===
input_df = pd.read_csv('datasets-13/new-data-for-test/rows-60-from-20240503/rows-60-from-20240503.csv',
                       sep='\t').dropna()
input_scaled = scaler.transform(
    input_df[['<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>']])
input_sequence = np.expand_dims(input_scaled, axis=0)  # shape: (1, 60, 5)

# === Predict the next 10 candles + Inverse scale ===
pred = best_model.predict(input_sequence)
prediction = close_scaler.inverse_transform(pred)

# plot section

In [ ]:
import sys

sys.path.insert(1, '../utils')
import forex_plot_utils
import os

# PARAMETERS
csv1_path = 'datasets-13/new-data-for-test/rows-60-from-20240503/latest-4-for-history.csv'
csv3_path = 'datasets-13/new-data-for-test/rows-60-from-20240503/after.csv'
plot_title = 'Actual vs Predicted Forex Closing Prices'
output_plot_path = None  # e.g., 'output.png'

# LOAD DATA FROM CSVS
historical_df = forex_plot_utils.load_csv_with_datetime(csv1_path) if os.path.exists(csv1_path) else None
actual_future_df = forex_plot_utils.load_csv_with_datetime(csv3_path) if os.path.exists(csv3_path) else None

# LOAD DATA FROM PREDICTION

# Combine <DATE> and <TIME> columns into a datetime
input_df['DATETIME'] = pd.to_datetime(input_df['<DATE>'] + ' ' + input_df['<TIME>'])

last_timestamp = input_df['DATETIME'].iloc[-1]
datetime_index = pd.date_range(start=last_timestamp + pd.Timedelta(hours=1), periods=len(prediction[0]), freq='h')

# Create DataFrame
predicted_df = pd.DataFrame({'DATETIME': datetime_index, '<CLOSE>': prediction[0]})

In [ ]:
# PLOT
forex_plot_utils.plot_all_series(
    historical_df=historical_df,
    predicted_df=predicted_df,
    actual_future_df=actual_future_df,
    title=plot_title,
    output_path=output_plot_path
)

In [ ]:
from datetime import datetime
import os
import pandas as pd
import matplotlib.pyplot as plt

# === Create timestamp and paths ===
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_filename = f'model_{timestamp}.keras'
model_path = os.path.join('saved_models', model_filename)

# Directory to hold logs and extras
log_dir = os.path.join('saved_models', f'model_{timestamp}_logs')
os.makedirs(log_dir, exist_ok=True)

# === Save model ===
best_model.save(model_path)

# === Save training history ===
history_df = pd.DataFrame(history.history)
history_df.to_csv(os.path.join(log_dir, 'training_history.csv'), index=False)

# === Save training loss plot ===
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(log_dir, 'training_loss.png'))
plt.close()

# === Save model summary and final performance ===
with open(os.path.join(log_dir, 'model_log.txt'), 'w') as f:
    best_model.summary(print_fn=lambda x: f.write(x + '\n'))
    final_train_loss = history.history['loss'][-1]
    final_test_loss, final_test_mae = best_model.evaluate(X_test, y_test, verbose=0)
    f.write(f'\nFinal Training Loss: {final_train_loss:.6f}\n')
    f.write(f'Final Test Loss: {final_test_loss:.6f}\n')
    f.write(f'Final Test MAE : {final_test_mae:.6f}\n')
